In [ ]:
using Plots, Zygote, LsqFit, Roots

Versions [Pluto](https://github.com/vlc1/ene-4102c-td/blob/master/td3.jl) et [Jupyter](https://vlc1.github.io/ene-4102c/td3.ipynb) de ce notebook.

In [ ]:
fig = contour(
    -3:0.01:.5,
    -3:0.01:3,
    (x, y) -> (1 + x) ^ 2 + y ^ 2,
    levels = [1],
    aspect_ratio = :equal,
    color = :blue,
    lw = 2,
    title = "Domaines de stabilité des méthodes EE et RK2"
)
contour!(
    fig,
    -3:0.01:.5,
    -3:0.01:3,
    (x, y) -> (1 + x + (x ^ 2 - y ^ 2) / 2) ^ 2 + (y * (1 + x)) ^ 2,
    levels = [1],
    aspect_ratio = :equal,
    color = :red,
    lw = 2
)

# Domaine de stabilité

On a vu en cours que les domaines de stabilité des schémas explicit d'Euler et RK2 sont respectivement
$$
\left \{ z \in \mathbb{C} \left  / \left \vert 1 + z \right \vert \le 1 \right . \right \}
$$
et
$$
\left \{ z \in \mathbb{C} \left  / \left \vert 1 + z + z ^ 2 / 2 \right \vert \le 1 \right . \right \}.
$$

1. Définir la raison $\overline{\sigma} \left ( z \right )$ ($z = \lambda \tau$) pour la méthode de RK4.
1. Visualiser le domaine de stabilité de RK4 grâce à la fonction `contour` de la bibliothèque `Plots.jl`.
1. Dans le cas $\lambda \in \mathbb{R} ^ -$, trouver le pas de temps maximal de RK4 (en fonction de $\lambda$). On utilisera la fonction `find_zero` de la bibliothèque `Roots.jl` (à installer).
1. Vérifier qu'au delà de ce pas de temps, cette méthode devient instable.


# Ordre de convergence

On rappelle que l'erreur d'un schéma à l'instant $t _ n$, définie par
$$
\epsilon = y_n - y \left ( t_n \right ),
$$
peut s'écrire
$$
\epsilon = C \tau ^ p
$$
où $p$ dénote l'**ordre** de la méthode.

1. Montrer que $\ln \epsilon$ est une fonction affine de $\ln \tau$.
1. En réutilisant le code du TD2 (voir ci-dessous), pour chaque schéma, calculer l'erreur à un temps donné (par exemple, ``s = 1``) en utilisant plusieurs pas de temps. On pourra s'inspirer du code suivant :
```julia
Δ = [0.125 / 2 ^ i for i in 5:-1:1]
methods = [explicit, implicit, midpoint, rk2, rk4]
errors = Dict(method => Float64[] for method in methods)
for method in methods
	pb = Problem(method, linear)
	for δ in Δ
		T, Y = integrate(pb, y, t, δ, s)
		append!(errors[method], abs(Y[end] - solution(T[end])))
	end
end
```
3. Visualiser l'erreur en fonction du pas de temps grâce aux fonctions `scatter`/`scatter!` en échelle logarithmique (`scale = :log`).
4. Utiliser le logarithme de ces valeurs ainsi que la méthode des moindres carrés, implémentée par la fonction `curve_fit` de `LsqFit.jl` (à installer), pour estimer les paramètres de la fonction suivante : `e(τ, p) = p[1] .+ p[2] .* τ`.


In [ ]:
function newton(f, x, p...)
    r = f(x, p...)
    while abs(r) > √eps(r)
        x -= r / first(gradient(f, x, p...))
        r = f(x, p...)
    end
    x, r
end

In [ ]:
struct Problem{F, G}
    scheme::F
    model::G
end

In [ ]:
(this::Problem)(var...) = this.scheme(this.model, var...)

In [ ]:
function integrate(problem, y, t, τ, s)
    T = [t]
    Y = [y]

	while t < (1 - √eps(t)) * s
        y, _ = newton(problem, y, y, τ, t)
        t += τ
        
        push!(Y, y)
        push!(T, t)
    end

    T, Y
end

In [ ]:
explicit(f, x, y, τ, t) = x - y - τ * f(t, y)

In [ ]:
implicit(f, x, y, τ, t) = x - y - τ * f(t + τ, x)

In [ ]:
trapezoidal(f, x, y, τ, t) = x - y - τ * (f(t, y) + f(t + τ, y)) / 2

In [ ]:
midpoint(f, x, y, τ, t) = x - y - τ * f(t + τ / 2, (x + y) / 2)

In [ ]:
function rk2(f, x, y, τ, t)
    z = y + τ * f(t, y) / 2
    x - y - τ * f(t + τ / 2, z)
end

In [ ]:
function rk4(f, x, y, τ, t)
	k₁ = τ * f(t, y)
	k₂ = τ * f(t + τ / 2, y + k₁ / 2)
	k₃ = τ * f(t + τ / 2, y + k₂ / 2)
	k₄ = τ * f(t + τ, y + k₃)
	x - y - (k₁ + 2k₂ + 2k₃ + k₄) / 6
end

In [ ]:
linear(t, y, λ = -1) = λ * y

In [ ]:
solution(t, λ = -1, y₀ = 1., t₀ = 0.) = exp(λ * (t - t₀)) * y₀

In [ ]:
problem = Problem(rk2, linear)

In [ ]:
y, t = 1.0, 0.

In [ ]:
τ, s = 0.1, 1.

In [ ]:
T, Y = integrate(problem, y, t, τ, s)